In [385]:
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from string import punctuation

In [386]:
def change_dual_words(word):
    first=''
    second=''
    def get_correct_word(w):
        if w=="sha":
            return("shall")
        elif w=="wo":
            return("will")
        else:
            return(w)
    word=word.lower()
    if word.endswith("n't"):
        first=get_correct_word(word[:word.index("n't")])
        second="not"
    elif word.endswith("'ll"):
        first=get_correct_word(word[:word.index("'ll")])
        second="will"
    elif word.endswith("'ve"):
        first=get_correct_word(word[:word.index("'ve")])
        second="have"
    elif word.endswith("'re"):
        first=get_correct_word(word[:word.index("'re")])
        second="are"
    elif word.endswith("'d"):
        first=get_correct_word(word[:word.index("'d")])
        second="did"
    elif word.endswith("'s"):
        first=get_correct_word(word[:word.index("'s")])
        second="is"
    else:
        first=word
        second="<allclear>"
    return((first,second))

In [406]:
def clean_corpus(word_lst):
    processed_lst=['<s>']
    for idx in range(0,len(word_lst)):
        if word_lst[idx] in ('.','?','!'):
#             processed_lst.append(word_lst[idx].lower())
            processed_lst.append('</s>')
            processed_lst.append('<s>')
        else:
            if word_lst[idx] not in punctuation:
                dual_words=change_dual_words(word_lst[idx].lower())
                if dual_words[1]=='<allclear>':
                    processed_lst.append(dual_words[0])
                else:
                    processed_lst.append(dual_words[0])
                    processed_lst.append(dual_words[1])
#     processed_lst.append('</s>')
    return(processed_lst)

In [491]:
bcorp_words=brown.words()
bcorp_words_cln=clean_corpus(bcorp_words)
bcorp_tot_words=len(bcorp_words_cln)
bcorp_freq_dist=nltk.FreqDist(word for word in bcorp_words_cln)
bigrams = ngrams(bcorp_words_cln,2)
# bcorp_bifreq_dist = nltk.ConditionalFreqDist(bigrams)
bcorp_bifreq_dist = nltk.FreqDist(bigrams)
trigrams = ngrams(bcorp_words_cln,3)
bcorp_trifreq_dist = nltk.FreqDist(trigrams)

In [523]:
def unigram_prob(word):
    try:
        return bcorp_freq_dist[word]/bcorp_tot_words
    except ZeroDivisionError as ze:
        return 0

In [524]:
def bigram_prob(bigram): 
#     print(bigram,bigram[0])
    try:
        return bcorp_bifreq_dist[bigram]/bcorp_freq_dist[bigram[0]]
    except ZeroDivisionError as ze:
        return 0

In [535]:
def trigram_prob(trigram): 
    try:
#         print(trigram,trigram[:2])
        return(bcorp_trifreq_dist[trigram]/bcorp_bifreq_dist[trigram[:2]])
    except ZeroDivisionError as ze:
        return 0

In [536]:
def get_sentence_prob(sentence):
    word_tokens=nltk.word_tokenize(sentence)
    word_lst=clean_corpus(word_tokens)
#     print(word_lst)
    sents_bigrams = ngrams(word_lst,2)
#     for x in sents_bigrams:print(x)
#     bigram_probs=[bigram_prob(gram) if bigram_prob(gram)!=0 else 1 for gram in sents_bigrams]
    bigram_probs=[0.8*bigram_prob(gram)+0.2*unigram_prob(gram[1]) for gram in sents_bigrams]
#     print(bigram_probs)
    prob=bigram_probs[0]
    for idx in range(1,len(bigram_probs)):
        prob=prob*bigram_probs[idx]
    print('Bigram Probabiltiy of sentense is %1.50f'%prob)
    
    sents_trigrams = ngrams(word_lst,3)
    trigram_probs=[0.6*trigram_prob(gram)+0.3*bigram_prob(gram[1:])+0.1*unigram_prob(gram[2]) for gram in sents_trigrams]
#     print(trigram_probs)
    prob=trigram_probs[0]
    for idx in range(1,len(trigram_probs)):
        prob=prob*trigram_probs[idx]
        
    print('Trigram Probabiltiy of sentense is %1.50f'%prob)
#     return("%1.10f"%(prob))

In [537]:
# get_sentence_prob('The jury further said in term-end presentments that the City Executive Committee which had over-all charge of the election deserves the praise and thanks of the City of Atlanta for the manner in which the election was conducted')
get_sentence_prob('The jury further said in term-end')

Bigram Probabiltiy of sentense is 0.00000000000000252612273714125053038671592814901338
Trigram Probabiltiy of sentense is 0.00000006762011110903911873149543755243118248188239


In [463]:
# 0.00000014153932978946989512764541817302355042329509
# 0.000000000234795896286408613828
# 0.000000000337864025877416325668
# 0.000000000391489835738799516328